In [67]:
import numpy as np
import os, csv
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

In [100]:
f = open('node_information.csv', 'r')
lines = f.readlines()
text_dict = {}
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("english")
for line in lines:
    line_info = line.strip().split(',')
    text = tokenizer.tokenize(line_info[-1])
    text = [t for t in text if t.isalpha()]
    text = [stemmer.stem(t) for t in text]
    text_dict[line_info[0]] = ' '.join(text)

In [101]:
print len(text_dict)

27770


In [102]:
text_dict[text_dict.keys()[0]]

u'we give the descript of discret moduli space d m s mcdisc introduc in cite in term of a discret de rham cohomolog for each moduli space mgn of a genus g n be the number of punctur we demonstr that intersect indic cohomolog class calcul for d m s coincid with the one for the continuum moduli space mc compactifi by delign and mumford procedur to show it we use a matrix model techniqu the kontsevich matrix model is a generat function for these indic in the continuum case and the matrix model with the potenti n alpha tr bigl fr l x l x log x x bigr is the one for d m s in the last case the effect of reduct becom relev but we use the stratif procedur in order to express integr over open space mdisc in term of intersect indic which are to be calcul on compactifi space the coincid of the cohomolog class for both continuum and d m s model enabl us to propos the exist of a quantum group structur on d m s then d m s are noth but cyclic except represent of a quantum group relat to a moduli spac

In [103]:
keys = []
corpus = []
for id, text in sorted(text_dict.iteritems(), key=lambda t: int(t[0])):
    keys.append(id)
    corpus.append(text)

In [104]:
print len(keys), len(corpus)

27770 27770


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, max_df=0.7, stop_words = 'english')

In [106]:
tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names() 

In [107]:
print len(feature_names)
tfidf_dense = tfidf_matrix.todense()

15764


In [114]:
tfidf_dense = np.asarray(tfidf_dense.tolist())
type(tfidf_dense)

numpy.ndarray

In [115]:
print tfidf_dense.shape

(27770, 15764)


In [111]:
np.savetxt('summary_tfidf.txt', tfidf_dense)

In [117]:
tfidf_dict = {}
i = 0
for item in keys:
    tfidf_dict[str(item)] = tfidf_dense[i]
    i+=1

In [130]:
f = open('training_set.txt')
lines = f.readlines()

In [134]:
from scipy import spatial

In [141]:
tfidf_feat = {}
for line in lines:
    entry = line.split()
    v1 = tfidf_dict[entry[0]]
    v2 = tfidf_dict[entry[1]]
    cosine = 1 - spatial.distance.cosine(v1, v2)
    tfidf_feat[(entry[0], entry[1])] = cosine
#     tfidf_feat.append((v1, v2, cosine))

In [142]:
import cPickle
cPickle.dump(tfidf_feat, open('tfidf_features.p', 'wb')) 

In [143]:
obj = cPickle.load(open('tfidf_features.p', 'rb'))

In [144]:
type(obj)

dict

In [145]:
f = open('testing_set.txt')
lines = f.readlines()
tfidf_feat_test = {}
for line in lines:
    entry = line.split()
    v1 = tfidf_dict[entry[0]]
    v2 = tfidf_dict[entry[1]]
    cosine = 1 - spatial.distance.cosine(v1, v2)
    tfidf_feat_test[(entry[0], entry[1])] = cosine

In [146]:
cPickle.dump(tfidf_feat_test, open('tfidf_features_test.p', 'wb')) 

In [147]:
len(tfidf_feat_test)

32648